In [5]:
import numpy as np
import psycopg2
import pandas as pd
import os 


db_params = {
    'dbname': 'eg_valorant',
    'user': 'udaya',
    'password': '&ETa$Tdj9z7msLo#',
    'host': '216.151.12.162',
    'port': '4928'
}



connection = psycopg2.connect(**db_params)
cursor = connection.cursor()


In [6]:

def list_tables(schema_name):
    try:
        # Connect to PostgreSQL
        connection = psycopg2.connect(**db_params)
        cursor = connection.cursor()

        # Query to get all tables in the specified schema
        table_query = '''
        SELECT table_name
        FROM information_schema.tables
        WHERE table_schema = %s;
        '''
        cursor.execute(table_query, (schema_name,))

        tables = cursor.fetchall()
        print(f"Available tables in the schema '{schema_name}':")
        for table in tables:
            print(table[0])

    except Exception as error:
        print(f"Error: {error}")
    finally:
        # Close the cursor and connection
        if cursor:
            cursor.close()
        if connection:
            connection.close()
        print("PostgreSQL connection is closed.")
schema_name = 'grid_valorant_hpe'
list_tables(schema_name)


Available tables in the schema 'grid_valorant_hpe':
event_damage
event_game_end
event_kills
data_health
grid_games
riot_player_combat
riot_player_game_stats
snapshots
freeze_end_inventory
grid_game_character
grid_game_round
grid_player_round
grid_series
riot_games
PostgreSQL connection is closed.


In [7]:
#get game_ids of EG Team


def get_dataframe_for_team_eg_by_game_ids(db_params, game_ids):
    try:
        # Connect to PostgreSQL
        connection = psycopg2.connect(**db_params)
        cursor = connection.cursor()

        # Create a string of game IDs for the SQL IN clause
        game_ids_str = ', '.join([f"'{game_id}'" for game_id in game_ids])

        # Query to get rows where Team is 'Eg' and game_id is in the provided list
        rows_query = f'''
        SELECT * 
        FROM grid_valorant_hpe.snapshots
        WHERE team = 'EG' AND game_id IN ({game_ids_str});
        '''
        cursor.execute(rows_query)

        # Fetch column names
        colnames = [desc[0] for desc in cursor.description]

        # Fetch rows
        rows = cursor.fetchall()

        # Convert to DataFrame
        df = pd.DataFrame(rows, columns=colnames)

        return df

    except Exception as error:
        print(f"Error fetching rows for Team 'Eg': {error}")
        return pd.DataFrame()
    finally:
        if cursor:
            cursor.close()
        if connection:
            connection.close()


In [8]:


# Function to query the database and return the result as a DataFrame
def query_to_dataframe(query, db_params):
    try:
        connection = psycopg2.connect(**db_params)
        df = pd.read_sql_query(query, connection)
        return df
    except Exception as error:
        print(f"Error: {error}")
    finally:
        if connection:
            connection.close()

schema_name = 'grid_valorant_hpe'

# Queries to fetch data from the respective tables
fnI_query = f'SELECT * FROM {schema_name}.freeze_end_inventory'
grid_series_query = f'SELECT * FROM {schema_name}.grid_games'
riot_games_query = f'SELECT * FROM {schema_name}.riot_games'
grid_game_character_query = f'SELECT * FROM {schema_name}.grid_game_character'
# Fetch data from the database
FnI = query_to_dataframe(fnI_query, db_params)
grid_series = query_to_dataframe(grid_series_query, db_params)
riot_games = query_to_dataframe(riot_games_query, db_params)
grid_game_character = query_to_dataframe(grid_game_character_query, db_params)

# Perform the merging operations
merged_df = pd.merge(grid_series, riot_games, on=['series_id', 'series_sequence'])
fnig = pd.merge(FnI, merged_df, on='game_id')
fdf = pd.merge(fnig, grid_game_character, on=['game_id', 'player_id', 'team_id'])
## data with inverntory_values with roit game ids 
iv = fdf[['round_num', 'riot_game_id', 'player', 'inventory_value']].copy().rename(columns={"riot_game_id": "game_id"})



def get_unique_game_ids():
    try:
        # Connect to PostgreSQL
        connection = psycopg2.connect(**db_params)
        cursor = connection.cursor()

        # Query to get unique game IDs
        unique_game_ids_query = '''
        SELECT DISTINCT game_id FROM grid_valorant_hpe.snapshots;
        '''
        cursor.execute(unique_game_ids_query)

        # Fetch unique game IDs
        unique_game_ids = cursor.fetchall()
        unique_game_ids = [gameid[0] for gameid in unique_game_ids]

        return unique_game_ids

    except Exception as error:
        print(f"Error fetching unique game IDs: {error}")
        return []
    finally:
        if cursor:
            cursor.close()
        if connection:
            connection.close()

def get_rows_by_game_id(game_id):
    try:
        # Connect to PostgreSQL
        connection = psycopg2.connect(**db_params)
        cursor = connection.cursor()

        # Query to get rows for a given game ID
        rows_by_game_id_query = '''
        SELECT * FROM grid_valorant_hpe.snapshots
        WHERE game_id = %s;
        '''
        cursor.execute(rows_by_game_id_query, (game_id,))

        # Fetch rows
        rows = cursor.fetchall()

        # Get column names
        colnames = [desc[0] for desc in cursor.description]

        # Create a DataFrame
        df = pd.DataFrame(rows, columns=colnames)

        return df

    except Exception as error:
        print(f"Error fetching rows for game ID {game_id}: {error}")
        return pd.DataFrame()
    finally:
        if cursor:
            cursor.close()
        if connection:
            connection.close()

unique_game_ids = get_unique_game_ids()
print("Unique game IDs:", len(unique_game_ids))

/tmp/ipykernel_2156238/3319575763.py:5: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, connection)
/tmp/ipykernel_2156238/3319575763.py:5: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, connection)
/tmp/ipykernel_2156238/3319575763.py:5: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, connection)
/tmp/ipykernel_2156238/3319575763.py:5: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sq

Unique game IDs: 321


In [9]:


def calculate_team_consequences(df):
    df_sorted = df.sort_values(by=['game_id', 'round_num', 'team', 'seconds'])
    df_sorted['kill_c'] = 0
    df_sorted['death_c'] = 0
    df_sorted['damage_dealt_c'] = 0
    df_sorted['damage_taken_c'] = 0

    for index, row in df_sorted.iterrows():
        window_start = row['seconds']
        window_end = window_start + 10

        subsequent_events = df_sorted[
            (df_sorted['game_id'] == row['game_id']) &
            (df_sorted['team'] == row['team']) &
            (df_sorted['round_num'] == row['round_num']) &
            (df_sorted['seconds'] > window_start) &
            (df_sorted['seconds'] <= window_end)
        ]

        df_sorted.loc[index, 'kill_c'] = subsequent_events['kill_change'].sum()
        df_sorted.loc[index, 'death_c'] = subsequent_events['death_change'].sum()
        df_sorted.loc[index, 'damage_dealt_c'] = subsequent_events['damage_dealt'].sum()
        df_sorted.loc[index, 'damage_taken_c'] = subsequent_events['damage_taken'].sum()

    return df_sorted


def calculate_player_consequences(df):
    df_sorted = df.sort_values(by=['game_id', 'round_num', 'player', 'seconds'])
    df_sorted['player_kill_c'] = 0
    df_sorted['player_death_c'] = 0
    df_sorted['player_damage_dealt_c'] = 0
    df_sorted['player_damage_taken_c'] = 0

    for index, row in df_sorted.iterrows():
        window_start = row['seconds']
        window_end = window_start + 10

        subsequent_events = df_sorted[
            (df_sorted['game_id'] == row['game_id']) &
            (df_sorted['player'] == row['player']) &
            (df_sorted['round_num'] == row['round_num']) &
            (df_sorted['seconds'] > window_start) &
            (df_sorted['seconds'] <= window_end)
        ]

        df_sorted.loc[index, 'player_kill_c'] = subsequent_events['kill_change'].sum()
        df_sorted.loc[index, 'player_death_c'] = subsequent_events['death_change'].sum()
        df_sorted.loc[index, 'player_damage_dealt_c'] = subsequent_events['damage_dealt'].sum()
        df_sorted.loc[index, 'player_damage_taken_c'] = subsequent_events['damage_taken'].sum()

    return df_sorted


def calculate_opponent_team_consequences(df):
    df_sorted = df.sort_values(by=['game_id', 'round_num', 'team', 'seconds'])
    df_sorted['opponent_kill_c'] = 0
    df_sorted['opponent_death_c'] = 0
    df_sorted['opponent_damage_dealt_c'] = 0
    df_sorted['opponent_damage_taken_c'] = 0

    for index, row in df_sorted.iterrows():
        window_start = row['seconds']
        window_end = window_start + 10

        subsequent_events = df_sorted[
            (df_sorted['game_id'] == row['game_id']) &
            (df_sorted['team'] != row['team']) &
            (df_sorted['round_num'] == row['round_num']) &
            (df_sorted['seconds'] > window_start) &
            (df_sorted['seconds'] <= window_end)
        ]

        df_sorted.loc[index, 'opponent_kill_c'] = subsequent_events['kill_change'].sum()
        df_sorted.loc[index, 'opponent_death_c'] = subsequent_events['death_change'].sum()
        df_sorted.loc[index, 'opponent_damage_dealt_c'] = subsequent_events['damage_dealt'].sum()
        df_sorted.loc[index, 'opponent_damage_taken_c'] = subsequent_events['damage_taken'].sum()

    return df_sorted


def get_event_game_end_by_game_id(riot_game_id, db_params):
    schema_name = 'grid_valorant_hpe'
    query = f'SELECT * FROM {schema_name}.event_game_end WHERE riot_game_id = %s'
    return query_to_dataframe_event_damage(query, (riot_game_id,), db_params)

def query_to_dataframe_event_damage(query, params, db_params):
    try:
        connection = psycopg2.connect(**db_params)
        df = pd.read_sql_query(query, connection, params=params)
        return df
    except Exception as error:
        print(f"Error: {error}")
    finally:
        if connection:
            connection.close()

def get_event_damage_by_game_id(game_id, db_params):
    schema_name = 'grid_valorant_hpe'
    query = f'SELECT * FROM {schema_name}.event_damage WHERE game_id = %s'
    return query_to_dataframe_event_damage(query, (game_id,), db_params)



In [40]:

# uncomment to get gameids of EG Team 
# df=get_dataframe_for_team_eg_by_game_ids(db_params, unique_game_ids)
#eg_game_ids=list(df["game_id"].unique())

In [10]:
gids=unique_game_ids.copy()  # all the unique game_ids 
#gids=[g for g in unique_game_ids if g not in eg_game_ids]
print("Total number of gameids :",len(gids))

Total number of gameids : 321


In [ ]:
output_file = './eg_train.csv'

#loop through each game_id and process data
for game_id in gids: #[60:70]:
    try:
        df = get_rows_by_game_id(game_id)
        df_s = df.sort_values(by='event_time')
        df_unique = df_s.drop_duplicates(subset=['game_id', 'round_num', 'clock_time', 'player'], keep='last')
        
        data_with_inventory = pd.merge(iv, df_unique, on=['game_id', 'player', 'round_num'])
        data_sorted = data_with_inventory.sort_values(by=['game_id', 'round_num', 'event_num'])
    
        # Calculate kill and death changes
        data_sorted['kill_change'] = data_sorted.groupby(['game_id', 'player',"round_num"])['kills'].diff().fillna(0).clip(lower=0)
        data_sorted['death_change'] = data_sorted.groupby(['game_id', 'player',"round_num"])['deaths'].diff().fillna(0).clip(lower=0)
        
        # Determine if player is alive
        data_sorted['is_alive'] = (data_sorted['hp'] > 0).astype(int)
        
        # Set inventory_value to 0 where hp is 0
        data_sorted.loc[data_sorted['hp'] == 0, 'inventory_value'] = 0
    
        # Calculate team alive count and health for our team
        data_sorted['our_team_alive'] = data_sorted.groupby(['game_id', 'round_num', 'event_num', 'team'])['is_alive'].transform('sum')
        data_sorted['our_team_health'] = data_sorted.groupby(['game_id', 'round_num', 'event_num', 'team'])['hp'].transform('sum')
        
        # Calculate total inventory value for each team at each event
        data_sorted['team_inventory_value'] = data_sorted.groupby(['game_id', 'round_num', 'event_num', 'team'])['inventory_value'].transform('sum')
        data_sorted['spike_event'] = data_sorted['spike_planted'].apply(lambda x: 'post-plant' if x else 'pre-plant')
    
        team_a = data_sorted[data_sorted['team'] == data_sorted["team"].unique()[0]].copy()
        team_b = data_sorted[data_sorted['team'] == data_sorted["team"].unique()[1]].copy()
    
        opponent_stats_a = team_b[['game_id', 'round_num', 'event_num', 'team', 'our_team_alive', 'our_team_health', 'team_inventory_value']].drop_duplicates(subset=['game_id', 'round_num', 'event_num'])
        opponent_stats_a.rename(columns={'team': 'opponent_team',
                                         'our_team_alive': 'opponent_team_alive',
                                         'our_team_health': 'opponent_team_health',
                                         'team_inventory_value': 'opponent_team_inventory_value'}, inplace=True)
    
        opponent_stats_b = team_a[['game_id', 'round_num', 'event_num', 'team', 'our_team_alive', 'our_team_health', 'team_inventory_value']].drop_duplicates(subset=['game_id', 'round_num', 'event_num'])
        opponent_stats_b.rename(columns={'team': 'opponent_team',
                                         'our_team_alive': 'opponent_team_alive',
                                         'our_team_health': 'opponent_team_health',
                                         'team_inventory_value': 'opponent_team_inventory_value'}, inplace=True)
    
        team_a = pd.merge(team_a, opponent_stats_a, on=['game_id', 'round_num', 'event_num'])
        team_b = pd.merge(team_b, opponent_stats_b, on=['game_id', 'round_num', 'event_num'])
        data_merged = pd.concat([team_a, team_b], ignore_index=True)
    
        for ability in ['ability1_base_charges', 'ability2_base_charges', 'grenade_base_charges', 'ultimate_base_charges']:
            data_merged[f'{ability}_change'] = data_merged.groupby(['game_id', 'round_num', 'player'])[ability].diff().fillna(0)
            data_merged[f'{ability}_gained'] = data_merged[f'{ability}_change'].clip(lower=0)
            data_merged[f'{ability}_used'] = (-data_merged[f'{ability}_change']).clip(lower=0)
    
        event_damage_df = get_event_damage_by_game_id(game_id, db_params)
    
        damage_df = event_damage_df[['game_id', 'round_num', 'event_num', 'attacker', 'attacker_team', 'victim', "victim_team", 'damage', "seconds"]]
        event_data = data_merged.copy()
        event_data['damage_dealt'] = np.nan
        event_data['damage_taken'] = np.nan
        damage_dealt_df = damage_df[damage_df['attacker_team'].isin(data_sorted['team'])]
        damage_taken_df = damage_df[damage_df['victim_team'].isin(data_sorted['team'])]
        attacker = damage_dealt_df.rename(columns={"attacker_team": "team", "damage": "damage_dealt", "attacker": "player"})
        victim = damage_taken_df.rename(columns={"victim_team": "team", "damage": "damage_taken", "victim": "player"})
        attacker = attacker[['game_id', 'round_num', 'team', 'damage_dealt', "seconds", "player", "event_num"]]
        victim = victim[['game_id', 'round_num', 'team', 'damage_taken', "seconds", "player", "event_num"]]
        damage_d = pd.concat([event_data, attacker], ignore_index=True)
        damage_t = pd.concat([damage_d, victim], ignore_index=True)
    
        damage_t['damage_dealt'].fillna(0, inplace=True)
        damage_t['damage_taken'].fillna(0, inplace=True)
    
        damage_df = calculate_team_consequences(damage_t)
        damage_df = calculate_player_consequences(damage_df)
        result_df = calculate_opponent_team_consequences(damage_df)
    
        round_result_data = get_event_game_end_by_game_id(game_id, db_params)
    
        round_result_data = round_result_data[round_result_data["round_result_code"] == "Defuse"].rename(columns={"riot_game_id": "game_id", "defuser_player": "player", "defuser_team": "team"})
        round_result_data["spike_diffused"] = True
        round_result = round_result_data[["game_id", "round_num", "team", "player", "spike_diffused"]]
    
        all_actions = pd.concat([result_df, round_result], ignore_index=True)
    
        grid_series_query = f'SELECT * FROM {schema_name}.grid_games'
        riot_games_query = f'SELECT * FROM {schema_name}.riot_games'
        grid_game_round_query = f'SELECT * FROM {schema_name}.grid_game_round'
    
        grid_series = query_to_dataframe(grid_series_query,db_params)
        riot_games = query_to_dataframe(riot_games_query,db_params)
        grid_game_round = query_to_dataframe(grid_game_round_query,db_params)
    
        riot_and_grid = pd.merge(grid_series, riot_games, on=['series_id', 'series_sequence'])
        grid_game_with_rid = pd.merge(grid_game_round, riot_and_grid, on=['game_id'])
    
        
        outcome = grid_game_with_rid[["riot_game_id", "team_id", "round_num", "won", "map_name", "side"]].copy().rename(columns={"riot_game_id": "game_id"})
    
        replacements = {
            'defender': 'defense',
            'attacker': 'attack'
        }
    
        outcome['side'] = outcome['side'].replace(replacements)
    
        time_events = pd.merge(all_actions, outcome, on=['game_id', "side", "round_num"])
    
        if not os.path.isfile(output_file):
            time_events.to_csv(output_file, index=False, mode='w')
        else:
            time_events.to_csv(output_file, index=False, mode='a', header=False)
    
        print(f"Processed and saved data for game_id: {game_id}")
    except Exception as e:
        print("unable to process game_id")
        print(f"An error occurred: {e}")
        pass
    
print("Processing complete. All data saved to:", output_file)


/tmp/ipykernel_2156238/2715202799.py:90: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, connection, params=params)
/tmp/ipykernel_2156238/2715202799.py:90: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, connection, params=params)
/tmp/ipykernel_2156238/3319575763.py:5: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, connection)
/tmp/ipykernel_2156238/3319575763.py:5: UserWarning: pandas only supports SQLAlchemy connectable (engine/connecti

Processed and saved data for game_id: 00ffbab0-1da4-48c8-bce9-b86db7cd9acb


/tmp/ipykernel_2156238/2715202799.py:90: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, connection, params=params)
